In [3]:
import os
import cv2
import numpy as np
from image_similarity_measures.quality_metrics import ssim, fsim, psnr
from shutil import copyfile
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor



# Function to calculate selected similarity measures
def calculate_similarity_metrics(img1, img2):
    # Ensure both images are (X, Y, 1)
    if len(img1.shape) == 2:
        img1 = np.expand_dims(img1, axis=-1)
    if len(img2.shape) == 2:
        img2 = np.expand_dims(img2, axis=-1)

    if img1.shape != img2.shape:
        raise ValueError(f"Image shapes are not the same: {img1.shape} vs {img2.shape}")

    metrics = {
        'ssim': ssim(img1, img2)
        ,'fsim': fsim(img1, img2)
        #,'psnr': psnr(img1, img2)
    }
    return metrics

# Function to convert white background to black
def convert_white_to_black_background(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold the image to create a mask of the white background
    _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    # Invert the mask to get the white background as 0
    mask_inv = cv2.bitwise_not(mask)
    # Create a black background image
    black_background = np.zeros_like(image)
    # Use the mask to combine the black background with the original image
    result = cv2.bitwise_or(black_background, black_background, mask=mask)
    result = cv2.bitwise_or(result, image, mask=mask_inv)
    return result

# Function to compute similarity score for a single pair
def compute_similarity_score(ct_image_path, hist_image_path):
    ct_img = cv2.imread(ct_image_path, cv2.IMREAD_GRAYSCALE)
    resized_ct_img = cv2.resize(ct_img, (256, 256))  # Resize to a manageable size
    resized_ct_img = np.expand_dims(resized_ct_img, axis=-1)  # Ensure the image is (256, 256, 1)

    hist_img = cv2.imread(hist_image_path)
    hist_img = convert_white_to_black_background(hist_img)  # Convert white background to black
    hist_img_gray = cv2.cvtColor(hist_img, cv2.COLOR_BGR2GRAY)
    resized_hist_img = cv2.resize(hist_img_gray, (256, 256))  # Resize to a manageable size

    metrics = calculate_similarity_metrics(resized_ct_img, resized_hist_img)
    avg_score = np.mean([metrics['ssim']
                         , metrics['fsim']
                         #, metrics['psnr']
                         ])  
    # Average of selected metric scores

    return (ct_image_path, hist_image_path, avg_score, metrics)

# Function to compute similarity scores for all pairs using parallel processing
def compute_similarity_scores(ct_images, histology_images):
    similarity_scores = []

    with ThreadPoolExecutor() as executor:
        futures = []
        for ct_image_path in ct_images:
            for hist_image_path in histology_images:
                futures.append(executor.submit(compute_similarity_score, ct_image_path, hist_image_path))

        for future in tqdm(futures, desc="Computing similarity scores"):
            similarity_scores.append(future.result())

    return similarity_scores

# Function to find the top n unique pairs
def find_top_n_pairs(similarity_scores, n):
    similarity_scores.sort(key=lambda x: x[2], reverse=True)  # Sort by average score in descending order
    selected_pairs = []
    used_ct_images = set()
    used_hist_images = set()

    for ct_image_path, hist_image_path, avg_score, metrics in similarity_scores:
        if len(selected_pairs) >= n:
            break
        if ct_image_path not in used_ct_images and hist_image_path not in used_hist_images:
            selected_pairs.append((ct_image_path, hist_image_path, avg_score, metrics))
            used_ct_images.add(ct_image_path)
            used_hist_images.add(hist_image_path)

    return selected_pairs

# Function to create pairs and save them in the paired directory
def save_paired_images(pairs):
    for i, (ct_image_path, hist_image_path, avg_score, metrics) in enumerate(pairs, start=1):
        ct_image_name = os.path.basename(ct_image_path)
        hist_image_name = os.path.basename(hist_image_path)

        paired_ct_path = os.path.join(paired_path, f"p{i}_ct_{ct_image_name}")
        paired_hist_path = os.path.join(paired_path, f"p{i}_hist_{hist_image_name}")

        copyfile(ct_image_path, paired_ct_path)
        copyfile(hist_image_path, paired_hist_path)

        print(f"Pair {i}: CT Image {ct_image_name} is paired with Histology Image {hist_image_name}")
        print(f"Metrics: {metrics}")

# List all CT and Histology images
def list_images(ct_path, histology_path):
    ct_images = [os.path.join(ct_path, img) for img in os.listdir(ct_path) if img.endswith(('.tif', '.png'))]
    histology_images = [os.path.join(histology_path, img) for img in os.listdir(histology_path) if img.endswith('.jpg')]
    return ct_images, histology_images

In [4]:
from random import sample
# Paths
ct_path = '../datasets/ct_hist_scans_preproc/trainA'
histology_path = "../\datasets/ct_hist_scans_preproc/trainB"
paired_path = "./example_Paired_Images/"
n = 5  # Specify the number of top pairs to select

# Ensure the paired images directory exists
if not os.path.exists(paired_path):
    os.makedirs(paired_path)
# Main execution
ct_images, histology_images = list_images(ct_path, histology_path)
ct_images = sample(ct_images, 25)
histology_images = sample(histology_images, 25)
similarity_scores = compute_similarity_scores(ct_images, histology_images)
top_pairs = find_top_n_pairs(similarity_scores, n)
save_paired_images(top_pairs)

Computing similarity scores: 100%|██████████| 625/625 [02:35<00:00,  4.01it/s]


Pair 1: CT Image E9-5_13-03_50kV_A_Rec_E9-5_13-03_50kV_rec00000945.png is paired with Histology Image s03r02s08.jpg
Metrics: {'ssim': 0.825040640468761, 'fsim': 0.3559147212323144}
Pair 2: CT Image E9-5_13-03_50kV_A_Rec_E9-5_13-03_50kV_rec00001023.png is paired with Histology Image s04r02s06.jpg
Metrics: {'ssim': 0.8074190239931639, 'fsim': 0.35897011710714216}
Pair 3: CT Image E9-5_13-03_50kV_A_Rec_E9-5_13-03_50kV_rec00000733.png is paired with Histology Image s17r02s03.jpg
Metrics: {'ssim': 0.8034057914939216, 'fsim': 0.335732362009839}
Pair 4: CT Image E9-5_13-03_50kV_A_Rec_E9-5_13-03_50kV_rec00000678.png is paired with Histology Image s19r02s05.jpg
Metrics: {'ssim': 0.7960104858494153, 'fsim': 0.3408164647893072}
Pair 5: CT Image E9-5_13-03_50kV_B_Rec_E9-5_13-03_50kV_rec00001881.png is paired with Histology Image s05r02s08.jpg
Metrics: {'ssim': 0.7719039532498676, 'fsim': 0.34635774172962924}


25*25 braucht 2:30 (ohne psnr)